In [1]:
import torch
import torchvision
import torch.optim as optim
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

In [2]:
class ResBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
        

class ResNet_CIFAR10(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.in_channels = 64
        self.hparams = hparams
        self.init_block = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.resblock_1 = self._make_resblock(64, 3, 1)
        self.resblock_2 = self._make_resblock(128, 4, 2)
        self.resblock_3 = self._make_resblock(256, 6, 2)
        self.resblock_4 = self._make_resblock(512, 3, 2)
        
        self.classifier = nn.Linear(512, 10)
        
        self.apply(self.init_weights)
    
    def init_weights(self, m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)

    
    def _make_resblock(self, channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(ResBlock(self.in_channels, channels, stride))
            self.in_channels = channels * ResBlock.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.init_block(x)
        out = self.resblock_1(out)
        out = self.resblock_2(out)
        out = self.resblock_3(out)
        out = self.resblock_4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        
        return out

    def general_step(self, batch, batch_idx):
        images, labels = batch
        out = self.forward(images)
        loss = F.cross_entropy(out, labels)

        preds = out.argmax(axis=1)
        acc = (labels == preds).float().mean()

        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, acc = self.general_step(batch, batch_idx)
        logs = {'train_loss': loss, 'train_acc': acc}
        return {'loss': loss, 'log': logs}

    def validation_step(self, batch, batch_idx):
        loss, acc = self.general_step(batch, batch_idx)
        logs = {'val_loss': loss, 'val_acc': acc}
        return {'val_loss': loss, 'val_acc': acc, 'log': logs}

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).sum()
        avg_acc /= len(outputs)
        logs = {'val_loss': avg_loss, 'val_acc': avg_acc}
        return {'val_loss': avg_loss, 'log': logs}

    def test_step(self, batch, batch_idx):
        loss, acc = self.general_step(batch, batch_idx)
        logs = {'test_loss': loss, 'test_acc': acc}
        return {'test_loss': loss, 'test_acc': acc, 'log': logs}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['test_acc'] for x in outputs]).sum()
        avg_acc /= len(outputs)
        logs = {'test_loss': avg_loss, 'test_acc': avg_acc}
        return {'test_loss': avg_loss, 'log': logs}
    
    def configure_optimizers(self):
        opt = optim.SGD(self.parameters(), lr=self.hparams['learning_rate'],  weight_decay=1e-5, momentum=0.9)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt)
        return [opt], [scheduler]

    def prepare_data(self):
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        cifar10_train = torchvision.datasets.CIFAR10(root='./data' , train=True, download=True, transform=transform)
        cifar10_test = torchvision.datasets.CIFAR10(root='./data' , train=False, download=True, transform=transform)
        
        self.train_set, self.val_set = random_split(cifar10_train, [40000, 10000])
        self.test_set = cifar10_test
        
    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.hparams["batch_size"], shuffle=True)
            
    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.hparams["batch_size"])
    
    def test_dataloader(self):
        return DataLoader(self.test_set)

In [3]:
hparams = {
    "batch_size": 256,
    "learning_rate": 1e-1,
    
}

model = ResNet_CIFAR10(hparams)

In [4]:
trainer = pl.Trainer(profiler=True, max_epochs=70, gpus=1)
trainer.fit(model)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
c:\users\thien\.virtualenvs\i2dl-a4dtsivt\lib\site-packages\pytorch_lightning\utilities\distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)


Files already downloaded and verified
Files already downloaded and verified



    | Name                    | Type        | Params
----------------------------------------------------
0   | init_block              | Sequential  | 1 K   
1   | init_block.0            | Conv2d      | 1 K   
2   | init_block.1            | BatchNorm2d | 128   
3   | init_block.2            | ReLU        | 0     
4   | resblock_1              | Sequential  | 221 K 
5   | resblock_1.0            | ResBlock    | 73 K  
6   | resblock_1.0.conv1      | Conv2d      | 36 K  
7   | resblock_1.0.bn1        | BatchNorm2d | 128   
8   | resblock_1.0.conv2      | Conv2d      | 36 K  
9   | resblock_1.0.bn2        | BatchNorm2d | 128   
10  | resblock_1.0.shortcut   | Sequential  | 0     
11  | resblock_1.1            | ResBlock    | 73 K  
12  | resblock_1.1.conv1      | Conv2d      | 36 K  
13  | resblock_1.1.bn1        | BatchNorm2d | 128   
14  | resblock_1.1.conv2      | Conv2d      | 36 K  
15  | resblock_1.1.bn2        | BatchNorm2d | 128   
16  | resblock_1.1.shortcut   | Sequential  |



Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_train_start      	|  0.016          	|  0.016          
on_epoch_start      	|  0.0020143      	|  0.141          
get_train_batch     	|  0.052848       	|  584.5          
on_batch_start      	|  1.6652e-05     	|  0.183          
model_forward       	|  0.023309       	|  256.16         
model_backward      	|  0.023447       	|  257.68         
on_after_backward   	|  2.9117e-06     	|  0.032          
optimizer_step      	|  0.024964       	|  274.35         
on_batch_end        	|  0.001787       	|  19.639         
on_epoch_end        	|  0.0            	|  0.0            
on_train_end        	|  0.015          	|  0.015          



1

In [5]:
trainer.test(model)

Files already downloaded and verified
Files already downloaded and verified


--------------------------------------------------------------------------------
TEST RESULTS
{'test_acc': tensor(0.7646, device='cuda:0'),
 'test_loss': tensor(1.2924, device='cuda:0')}
--------------------------------------------------------------------------------



In [6]:
torch.save(model.state_dict(), './models/resnet34_cifar10.pth')